## Importing libraries

In [209]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from kkltk.kin_kir_stopwords import stopwords 
from gensim.models import Word2Vec
import torch
import random
import torch.nn as nn
import torch.optim as optim
import torchtext.vocab as vocab
import spacy
from torchtext import data
from torchtext.data.utils import get_tokenizer
from tqdm import tqdm
from torchtext.data import Field, LabelField, TabularDataset, BucketIterator
import re

## Stopwords set

In [129]:
stopset_kin = stopwords.words('kinyarwanda') 

## Dataset to csv

In [130]:
data = pd.read_csv('/Users/casarulez/Projects/DDSI/SubtaskB/train/multilingual_train.tsv',sep='\t')

In [131]:
data.drop('ID', axis=1, inplace=True)

In [132]:
data.tweet = data.tweet.str.replace('[^A-Za-z\s\’\-]+', '')
data.tweet = data.tweet.str.replace('[\n]+', '')
data.tweet = data.tweet.str.replace('^https?:\/\/.*[\r\n]*', '')

In [133]:
# Removing the stopwords
data['tweet'] = data['tweet'].apply(lambda x: ' '.join([item.lower() for item in str(x).split() if item not in stopset_kin]))

In [134]:
data.to_csv('/Users/casarulez/Projects/DDSI/SubtaskB/train/multilingual_train.csv', index=False)

## Importing raw dataset 

In [180]:
data = pd.read_csv('/Users/casarulez/Projects/DDSI/SubtaskB/train/multilingual_train.csv')
data

,tweet,label
0,if i dey enter your eye or you like me and fit...,positive
1,@user @user ndi igbo is ara di udi,negative
2,كاملين بيوكوم عنستفكوم من فوق شواية الا بنكيرا...,negative
3,samia atoa angalizo kuikabili saratani makamu ...,neutral
4,"5. dùnùndún, ọ̀jọ̀jọ̀, ____ ẹ̀bà, ____, àmàlà ...",neutral
...,...,...
63680,ትዊተር እንደ ዛሬ ደባሪ ሆኖ አያውቅም።ህዝቤ ወጥሮ ጭራውን እየቆላ ነው።,negative
63681,"ọ̀tàkìtì pọ́n'wọ́ lá, ò b'omi 'ẹ́nu fẹ́'ná jó,...",positive
63682,غير انا الي مجانيش دعم من هاذا التيم واقيل 💔 @...,negative
63683,ከነ ድንኳኑ ነዋ ለዛውም ???♂️?????? የሠርግ ይሁን የለቅሶ እሱ ይወቀው,neutral


## Cleaning and saving

In [181]:
def clean_tweets(csv_input_path, csv_output_path):
    # Load data
    data = pd.read_csv(csv_input_path)
    
    # Ensure all entries in the 'tweet' column are strings
    data['tweet'] = data['tweet'].astype(str)
    
    # Remove symbols but keep letters and spaces
    data.tweet = data.tweet.str.replace(r'[^\w\s\’\-]+', '', regex=True)

    # Remove newline characters
    data.tweet = data.tweet.str.replace(r'\n', '', regex=True)

    # Remove URLs
    data.tweet = data.tweet.str.replace(r'http\S+|www\S+', '', regex=True)

    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)

    data.tweet = data.tweet.apply(lambda x: emoji_pattern.sub(r'', x))

    # Save the cleaned data to a new CSV file
    data.to_csv(csv_output_path, index=False)

In [182]:
clean_tweets('/Users/casarulez/Projects/DDSI/SubtaskB/train/multilingual_train.csv','/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_train.csv')

In [183]:
clean_tweets('/Users/casarulez/Projects/DDSI/SubtaskB/test/multilingual_test.csv','/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_test.csv')

## Generating word embeddings 

In [184]:
from gensim.models import Word2Vec

# Load the data
data_train = pd.read_csv('/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_train.csv')
data_test = pd.read_csv('/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_test.csv')

# Concatenate the data
data = pd.concat([data_train, data_test])

# Ensure 'tweet' and 'label' columns do not have missing values
data['tweet'] = data['tweet'].fillna('')
data['label'] = data['label'].fillna('')

# Create a combined 'whole_doc' column
data['whole_doc'] = data['tweet'] + ' ' + data['label'].astype(str)

# Prepare the sentences for Word2Vec
sentences = [row.split() for row in data['whole_doc'] if len(row)]
sentences = [[token.lower() for token in sentence if token] for sentence in sentences]

# Train the Word2Vec model
w2v_model = Word2Vec(sentences, window=5, min_count=5, sg=1, hs=1, vector_size=50)

# Save the word vectors
with open("/Users/casarulez/Projects/DDSI/SubtaskB/trained_embeddings/embeddings.txt", 'w') as output:
    for token in w2v_model.wv.key_to_index:
        vector_str = ' '.join(map(str, w2v_model.wv[token]))
        output.write(f"{token} {vector_str}\n")


## Loading the word embeddings

In [185]:
# Generate the custom embeddings
custom_embeddings = vocab.Vectors(name='/Users/casarulez/Projects/DDSI/SubtaskB/trained_embeddings/embeddings.txt',
                                  cache='/Users/casarulez/Projects/DDSI/SubtaskB/trained_embeddings/',
                                  unk_init=torch.Tensor.normal_)

In [186]:
# Ensure reproducibility
SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [187]:
import spacy
from torchtext import data

spacy_en = spacy.load('en_core_web_sm')

# Define fields to hold the data
LABEL = data.LabelField(dtype=torch.float)
TWEET = data.Field(tokenize='spacy', tokenizer_language='en_core_web_sm')

In [188]:
fields = [('label', LABEL), ('tweet', TWEET)]

## Loading cleaned train and test data 

In [205]:
train_data=pd.read_csv('/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_train.csv')
test_data=pd.read_csv('/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_test.csv')

In [206]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63685 entries, 0 to 63684
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   63685 non-null  object
 1   label   63685 non-null  object
dtypes: object(2)
memory usage: 995.2+ KB


In [207]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30210 entries, 0 to 30209
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   30210 non-null  object
 1   label   30210 non-null  object
dtypes: object(2)
memory usage: 472.2+ KB


### Dropping null values 

In [191]:
train_data.dropna(inplace=True)

In [192]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63685 entries, 0 to 63684
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   63685 non-null  object
 1   label   63685 non-null  object
dtypes: object(2)
memory usage: 995.2+ KB


In [193]:
test_data.dropna(inplace=True)

In [195]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30210 entries, 0 to 30210
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   30210 non-null  object
 1   label   30210 non-null  object
dtypes: object(2)
memory usage: 708.0+ KB


In [203]:
train_data.to_csv('/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_train.csv', index=False)

In [204]:
test_data.to_csv('/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_test.csv', index=False)

In [210]:
train_data = TabularDataset(
    path='/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_train.csv',
    format='csv',
    fields=fields
)

In [211]:
test_data = TabularDataset(
    path='/Users/casarulez/Projects/DDSI/SubtaskB/cleaned/cleaned_test.csv',
    format='csv',
    fields=fields
)

## Building vocabulary objects

In [212]:
# build the vocabulary
TWEET.build_vocab(train_data.tweet, max_size=15000, vectors=custom_embeddings)
LABEL.build_vocab(train_data.label)

## Defining the model 

In [228]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.content),
    device=device)


In [229]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim,
                 output_dim, n_layers, bidirectional, dropout):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers=n_layers,
                          bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, hidden = self.rnn(embedded)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        return self.fc(hidden.squeeze(0))

In [230]:
# Create the instance of the model
INPUT_DIM = len(TWEET.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

pretrained_embeddings = TWEET.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [237]:
print(model)

RNN(
  (embedding): Embedding(6, 50)
  (rnn): GRU(50, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=62616, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [222]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # Round predictions to the closest integer
    rounded_preds = torch.max(preds, -1)[1]
    correct = (rounded_preds == y).float()  # convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [231]:
def train(model, iterator, optimizer, criterion):
    """ Training the model"""
    epoch_loss = 0
    epoch_acc = 0
    model.train()

    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.tweet).squeeze(1)  # Assuming tweet is the text data
        loss = criterion(predictions, batch.label.type(torch.cuda.LongTensor))
        acc = multiclass_accuracy(predictions, batch.label.type(torch.cuda.LongTensor))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


## Training 

In [232]:
# Number of epochs
N_EPOCHS = 10

# Loop over epochs
for epoch in range(N_EPOCHS):
    # Call the train function and get the loss and accuracy
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # Print training metrics
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% |')


RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.

In [236]:
!pip freeze > requirements.txt